In [466]:
import yfinance as yf
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from finta import TA
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import jaccard_score
import seaborn as sn
from tabulate import tabulate
from xgboost import XGBClassifier
from ta import add_all_ta_features
from sklearn.feature_selection import RFE
import xgboost as xgb

In [467]:
WINDOW = 16 # number of rows to look ahead to see what the price did
FETCH_INTERVAL = "60m"  # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
INTERVAL = '2y'     # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
symbol = 'AAPL'      # Symbol of the desired stock

# one day 16 rows of data

In [468]:
data = yf.download(  # or pdr.get_data_yahoo(...
        tickers = symbol,


        period = INTERVAL,

        interval = FETCH_INTERVAL,

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        # auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = True,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2019-08-21 04:00:00-04:00,52.775000,53.110000,52.775000,53.050000,53.050000,0
2019-08-21 05:00:00-04:00,53.045000,53.150000,53.045000,53.150000,53.150000,0
2019-08-21 06:00:00-04:00,53.152500,53.200000,53.045000,53.080000,53.080000,0
2019-08-21 07:00:00-04:00,53.112500,53.112500,53.000000,53.070000,53.070000,0
2019-08-21 08:00:00-04:00,53.050000,53.100000,53.000000,53.065000,53.065000,0
...,...,...,...,...,...,...
2021-08-20 13:30:00-04:00,147.912003,147.994995,147.550003,147.720001,147.720001,4405041
2021-08-20 14:30:00-04:00,147.714996,148.165894,147.559998,148.084793,148.084793,4758189
2021-08-20 15:30:00-04:00,148.087006,148.289993,147.945007,148.190002,148.190002,6188690


In [469]:
# data = pd.read_csv('C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\preprocess_new\\AAPL_2y_8_11_05_2021 09_00_35_full.csv', index_col='Datetime')
# data

In [470]:
data.rename(columns={"Close": 'close', "High": 'high', "Low": 'low', 'Volume': 'volume', 'Open': 'open'}, inplace=True)
data.head(10)
important_columns = ['open', 'high', 'low','close','volume']

In [471]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8377 entries, 2019-08-21 04:00:00-04:00 to 2021-08-20 17:00:00-04:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   open       8377 non-null   float64
 1   high       8377 non-null   float64
 2   low        8377 non-null   float64
 3   close      8377 non-null   float64
 4   Adj Close  8377 non-null   float64
 5   volume     8377 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 458.1 KB


In [472]:

def calculate_diffs(diff_number, col_name):
    new_col_name = f'{col_name}_{diff_number}'
    data[new_col_name] = data[col_name].diff(diff_number)

In [473]:
# for name in important_columns:
#     for i in range(1, 11):
#         calculate_diffs(i, name)
#
# data.head(10)

In [474]:
data = data.dropna()

In [475]:
# data
# data.set_index(pd.DatetimeIndex(data['Datetime']))
# data

In [476]:
# fig, ax = plt.subplots()
#
# data.plot(kind='line',x='Datetime',y='close',ax=ax)
#
# every_nth = 4
# for n, label in enumerate(ax.xaxis.get_ticklabels()):
#     if n % every_nth != 0:
#         label.set_visible(False)
#
# plt.xticks(rotation=90)
# plt.show()

In [477]:
# import seaborn as sns
# sns.set()
# ax = data['open'].plot()
# plt.xticks(rotation=90)
# plt.ylabel('Kurs akcji [$]', rotation=90)
# plt.xlabel('Czas')
# plt.legend(['Close'], ncol=1, loc='upper left');
# plt.show()
#
# fig = ax.get_figure()
# fig.savefig('asdf.png', dpi=300, bbox_inches="tight")
#

In [478]:
# data['close_pct'] = data['close'].pct_change()
# data.head()

In [479]:
data.describe()

,open,high,low,close,Adj Close,volume
count,8377.000000,8377.000000,8377.000000,8377.000000,8377.000000,8.377000e+03
mean,101.051622,101.468736,100.545858,101.056083,101.056083,3.809317e+06
std,29.983177,30.346019,29.813513,29.980169,29.980169,6.934562e+06
min,50.250000,50.467500,50.250000,50.357500,50.357500,0.000000e+00
25%,71.577500,71.949997,71.217500,71.625000,71.625000,0.000000e+00
50%,111.446251,111.962500,110.590000,111.410000,111.410000,0.000000e+00
75%,126.910004,127.250000,126.360000,126.915001,126.915001,5.486126e+06
max,151.410000,438.440000,151.290000,151.670000,151.670000,9.845401e+07


In [480]:
from ta import add_all_ta_features

def _get_indicator_data(data):

    data = add_all_ta_features(
        data, open="open", high="high", low="low", close="close", volume="volume", fillna=True)

    return data


In [481]:
data

,open,high,low,close,Adj Close,volume
Datetime,,,,,,
2019-08-21 04:00:00-04:00,52.775000,53.110000,52.775000,53.050000,53.050000,0
2019-08-21 05:00:00-04:00,53.045000,53.150000,53.045000,53.150000,53.150000,0
2019-08-21 06:00:00-04:00,53.152500,53.200000,53.045000,53.080000,53.080000,0
2019-08-21 07:00:00-04:00,53.112500,53.112500,53.000000,53.070000,53.070000,0
2019-08-21 08:00:00-04:00,53.050000,53.100000,53.000000,53.065000,53.065000,0
...,...,...,...,...,...,...
2021-08-20 13:30:00-04:00,147.912003,147.994995,147.550003,147.720001,147.720001,4405041
2021-08-20 14:30:00-04:00,147.714996,148.165894,147.559998,148.084793,148.084793,4758189
2021-08-20 15:30:00-04:00,148.087006,148.289993,147.945007,148.190002,148.190002,6188690


In [482]:
data.columns


Index(['open', 'high', 'low', 'close', 'Adj Close', 'volume'], dtype='object')

In [483]:
def create_class_column(row, lowest_threshold, higher_threshold):
    if row['close_shift'] - row['close'] > higher_threshold:
        return 1
    if row['close_shift'] - row['close'] < lowest_threshold:
        return -1
    else:
        return 0

In [484]:
data = _get_indicator_data(data)
data['close_shift'] = data.shift(-WINDOW)['close']
data

C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\ta\trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\ta\trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


,open,high,low,close,Adj Close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,close_shift
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-08-21 04:00:00-04:00,52.775000,53.110000,52.775000,53.050000,53.050000,0,0.000000e+00,0,0.000000,0.000000,...,0.000000,53.050000,0.000000,0.000000,0.000000,0.000000,-47.504397,0.000000,0.000000,53.3475
2019-08-21 05:00:00-04:00,53.045000,53.150000,53.045000,53.150000,53.150000,0,0.000000e+00,0,0.000000,0.000000,...,0.000000,53.094125,0.000000,0.000000,0.000000,0.000000,0.188501,0.188324,0.188501,53.2000
2019-08-21 06:00:00-04:00,53.152500,53.200000,53.045000,53.080000,53.080000,0,0.000000e+00,0,0.000000,0.000000,...,0.000000,53.087931,0.000000,0.000000,0.000000,0.000000,-0.131703,-0.131790,0.056550,52.9025
2019-08-21 07:00:00-04:00,53.112500,53.112500,53.000000,53.070000,53.070000,0,0.000000e+00,0,0.000000,0.000000,...,0.000000,53.080061,0.000000,0.000000,0.000000,0.000000,-0.018839,-0.018841,0.037700,53.0250
2019-08-21 08:00:00-04:00,53.050000,53.100000,53.000000,53.065000,53.065000,0,0.000000e+00,0,0.000000,0.000000,...,0.000000,53.073436,0.000000,0.000000,0.000000,0.000000,-0.009422,-0.009422,0.028275,53.1950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-20 13:30:00-04:00,147.912003,147.994995,147.550003,147.720001,147.720001,4405041,1.740461e+09,510276027,0.054954,481052.780475,...,1.250673,146.998141,0.695297,6.479453,-5.881436,12.360889,-0.133651,-0.133740,178.454291,NaN
2021-08-20 14:30:00-04:00,147.714996,148.165894,147.559998,148.084793,148.084793,4758189,1.743945e+09,515034216,0.033182,660295.049504,...,1.318058,147.190885,1.047283,6.175124,-3.470124,9.645248,0.246948,0.246644,179.141929,NaN
2021-08-20 15:30:00-04:00,148.087006,148.289993,147.945007,148.190002,148.190002,6188690,1.746547e+09,521222906,0.170270,658982.621843,...,1.283838,147.387734,1.333426,8.251022,-1.125895,9.376917,0.071047,0.071021,179.340250,NaN


In [485]:
higher_threshold = 1.5
lowest_threshold = -1.5
last_values_higher = []
last_values_lower = []
data['class_column'] = data.apply((lambda x: create_class_column(x, lowest_threshold, higher_threshold)), axis=1)
while True:
    # print(data['class_column'].value_counts())
    class_counts = data['class_column'].value_counts()
    if abs(class_counts[0] - class_counts[1]) < 15 and abs(class_counts[0] - class_counts[-1]) < 15:
        break

    if len(last_values_higher) == 3:
        last_values_higher.pop(0)
    if len(last_values_lower) == 3:
        last_values_lower.pop(0)

    last_values_higher.append(higher_threshold)
    last_values_lower.append(lowest_threshold)
    if class_counts[0] > class_counts[1]:
        higher_threshold -= 0.01
    if class_counts[0] > class_counts[-1]:
        lowest_threshold += 0.01
    if class_counts[0] < class_counts[1]:
        higher_threshold += 0.01
    if class_counts[0] < class_counts[-1]:
        lowest_threshold -= 0.01

    if higher_threshold in last_values_higher and lowest_threshold in last_values_lower:
        break
    # print(higher_threshold, lowest_threshold)
    data['class_column'] = data.apply((lambda x: create_class_column(x, lowest_threshold, higher_threshold)), axis=1)

data

,open,high,low,close,Adj Close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,close_shift,class_column
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-08-21 04:00:00-04:00,52.775000,53.110000,52.775000,53.050000,53.050000,0,0.000000e+00,0,0.000000,0.000000,...,53.050000,0.000000,0.000000,0.000000,0.000000,-47.504397,0.000000,0.000000,53.3475,0
2019-08-21 05:00:00-04:00,53.045000,53.150000,53.045000,53.150000,53.150000,0,0.000000e+00,0,0.000000,0.000000,...,53.094125,0.000000,0.000000,0.000000,0.000000,0.188501,0.188324,0.188501,53.2000,0
2019-08-21 06:00:00-04:00,53.152500,53.200000,53.045000,53.080000,53.080000,0,0.000000e+00,0,0.000000,0.000000,...,53.087931,0.000000,0.000000,0.000000,0.000000,-0.131703,-0.131790,0.056550,52.9025,0
2019-08-21 07:00:00-04:00,53.112500,53.112500,53.000000,53.070000,53.070000,0,0.000000e+00,0,0.000000,0.000000,...,53.080061,0.000000,0.000000,0.000000,0.000000,-0.018839,-0.018841,0.037700,53.0250,0
2019-08-21 08:00:00-04:00,53.050000,53.100000,53.000000,53.065000,53.065000,0,0.000000e+00,0,0.000000,0.000000,...,53.073436,0.000000,0.000000,0.000000,0.000000,-0.009422,-0.009422,0.028275,53.1950,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-20 13:30:00-04:00,147.912003,147.994995,147.550003,147.720001,147.720001,4405041,1.740461e+09,510276027,0.054954,481052.780475,...,146.998141,0.695297,6.479453,-5.881436,12.360889,-0.133651,-0.133740,178.454291,NaN,0
2021-08-20 14:30:00-04:00,147.714996,148.165894,147.559998,148.084793,148.084793,4758189,1.743945e+09,515034216,0.033182,660295.049504,...,147.190885,1.047283,6.175124,-3.470124,9.645248,0.246948,0.246644,179.141929,NaN,0
2021-08-20 15:30:00-04:00,148.087006,148.289993,147.945007,148.190002,148.190002,6188690,1.746547e+09,521222906,0.170270,658982.621843,...,147.387734,1.333426,8.251022,-1.125895,9.376917,0.071047,0.071021,179.340250,NaN,0


In [486]:
# Class divide
data['class_column'] = data.apply((lambda x: create_class_column(x, lowest_threshold, higher_threshold)), axis=1)

data['class_column'].value_counts()

 0    2817
 1    2802
-1    2758
Name: class_column, dtype: int64

In [487]:
filename_to_export = f'C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\results\\train_test\\{symbol}_{INTERVAL}_{WINDOW}_diff_{datetime.now().strftime("%d_%m_%Y %H_%M_%S")}_full.csv'
data.to_csv(filename_to_export, index=True)
print(filename_to_export)

C:\Users\exomat\Desktop\repo\magisterka_analiza\data\results\train_test\AAPL_2y_16_diff_20_08_2021 23_13_39_full.csv


In [488]:

# Class divide
data['class_column'].value_counts()

 0    2817
 1    2802
-1    2758
Name: class_column, dtype: int64

In [489]:
# del (data['close'])
del (data['close_shift'])
data = data.dropna()

In [490]:
data['class_column'].value_counts()


 0    2817
 1    2802
-1    2758
Name: class_column, dtype: int64

In [491]:
y = data['class_column']
features = [x for x in data.columns if x not in ['class_column']]
x = data[features]
# scaler = MinMaxScaler()
# x = pd.DataFrame(scaler.fit_transform(x.values), columns=x.columns, index=x.index)
ROWS_TO_PREDICT = 512
if ROWS_TO_PREDICT == 33:
    x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.3, shuffle=False)
else:
    x_train= x.iloc[:-ROWS_TO_PREDICT]
    y_train= y.iloc[:-ROWS_TO_PREDICT]
    x_test =x.iloc[-ROWS_TO_PREDICT:]
    y_test=y.iloc[-ROWS_TO_PREDICT:]

In [492]:
classifiers = dict()
classifiers['DecisionTreeClassifier 1'] = DecisionTreeClassifier(max_depth=15, random_state=0, criterion='gini',
                                                                 splitter='best')
classifiers['DecisionTreeClassifier 2'] = DecisionTreeClassifier(max_depth=40, random_state=0, criterion='gini',
                                                                 splitter='best')
classifiers['DecisionTreeClassifier 3'] = DecisionTreeClassifier(max_depth=10, random_state=0,criterion='gini',splitter='best')


classifiers['RandomForestClassifier 1'] = RandomForestClassifier(n_estimators=1000, max_depth=3, random_state=0,
                                                                 criterion='gini', n_jobs=-1)

classifiers['RandomForestClassifier 2'] = RandomForestClassifier(n_estimators=1000, max_depth=3, random_state=0,
                                                                 criterion='entropy', n_jobs=-1)

classifiers['RandomForestClassifier 3'] = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=0,
                                                                 criterion='entropy', n_jobs=-1)


classifiers['GradientBoostingClassifier 1'] = GradientBoostingClassifier(n_estimators=100, random_state=0,
                                                                         criterion='friedman_mse', max_depth=3,
                                                                         learning_rate=0.1)

classifiers['GradientBoostingClassifier 2'] = GradientBoostingClassifier(n_estimators=1000, random_state=0,
                                                                         criterion='friedman_mse', max_depth=3,
                                                                         learning_rate=0.5)


classifiers['XGBRFClassifier 1'] = xgb.sklearn.XGBRFClassifier(n_jobs=-1, max_depth=2, n_estimators=1000, eta=0.2)
classifiers['XGBRFClassifier 2'] = xgb.sklearn.XGBRFClassifier(n_jobs=-1, max_depth=6, n_estimators=1000, eta=0.3)

classifiers['XGBClassifier 1'] = xgb.XGBClassifier(nthread=-1, max_depth=3, n_estimators=1000, eta=0.2)
classifiers['XGBClassifier 2'] = xgb.XGBClassifier(nthread=-1, max_depth=3, n_estimators=1000, eta=0.3)

classifiers_boosted = dict()
classifiers_boosted['GradientBoostingClassifier 1S'] = GradientBoostingClassifier(n_estimators=100, random_state=0,
                                                                                  criterion='friedman_mse', max_depth=3,
                                                                                  learning_rate=0.1)
classifiers_boosted['SXGBClassifier 1'] = xgb.XGBClassifier(nthread=-1, max_depth=3, n_estimators=1000, eta=0.2)
classifiers_boosted['SXGBClassifier 2'] = xgb.XGBClassifier(nthread=-1, max_depth=3, n_estimators=1000, eta=0.3)
classifiers_boosted['SXGBClassifier 3'] = xgb.XGBClassifier(nthread=-1, max_depth=10, n_estimators=1000, eta=0.2)

In [493]:
def print_conf_matrix(test_y, predict, name):
    matrix = confusion_matrix(test_y, predict,  labels=[-1, 0, 1])
    print(matrix)
    ax= plt.subplot()
    sn.heatmap(matrix, annot=True, ax = ax) #annot=True to annotate cells

    # labels, title and ticks
    ax.set_xlabel('Predicted labels', color='white')
    ax.set_ylabel('True labels', color='white')
    ax.set_title(f'Confusion Matrix for {name}' , color='white')
    ax.xaxis.set_ticklabels(['-1','0', '1'], color='white')
    ax.yaxis.set_ticklabels(['-1','0', '1'], color='white')
    plt.show()

def train_model(model,train_x, train_y):
    model.fit(train_x, train_y)

In [494]:
predictions_train= dict()
score_train = dict()
predictions= dict()
score = dict()

In [495]:
# predictions_train= dict()
# score_train = dict()
# predictions= dict()
# score = dict()
# for k,v in classifiers.items():
#     print("Calculate: ", k)
#     train_model(v,x_train,y_train)
#     predictions_train[k] = v.predict(x_train)
#     score_train[k] = accuracy_score(y_train.values, predictions_train[k])
#     predictions[k] = v.predict(x_test)
#     score[k] = accuracy_score(y_test.values, predictions[k])
#     print('Score_train: ',  score_train[k] )
#     print('Score: ',  score[k] )
#     # print_conf_matrix(test_y, predictions[k], k)
#

In [496]:
# filename_to_export = f'C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\results\\train_test\\result_test_xrand_{symbol}_{WINDOW}_{ROWS_TO_PREDICT}_{datetime.now().strftime("%d_%m_%Y %H_%M_%S")}.csv'
# filename_to_export_train= f'C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\results\\train_test\\result_train_xrand_{symbol}_{WINDOW}_{ROWS_TO_PREDICT}_{datetime.now().strftime("%d_%m_%Y %H_%M_%S")}.csv'
#
# headers = ["Classifier type", "Accuracy"]
# score_df = pd.DataFrame(score.items(), columns=headers)
# print(tabulate(score_df, headers, tablefmt="psql"))
# score_df.to_csv(filename_to_export,mode='a', index=False, header=False)
# score_df_train = pd.DataFrame(score_train.items(), columns=headers)
# print(tabulate(score_df_train, headers, tablefmt="psql"))
# score_df_train.to_csv(filename_to_export_train,mode='a', index=False, header=False)

In [497]:
rfe = RFE(classifiers['XGBRFClassifier 6'],10)
fited = rfe.fit(x_train, y_train)
rfe

C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:13:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:14:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:28:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:28:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RFE(estimator=XGBRFClassifier(base_score=None, booster=None,
                              colsample_bylevel=None, colsample_bytree=None,
                              eta=0.6, gamma=None, gpu_id=None,
                              importance_type='gain',
                              interaction_constraints=None, max_delta_step=None,
                              max_depth=3, min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=1000,
                              n_jobs=-1, num_parallel_tree=None,
                              objective='binary:logistic', random_state=None,
                              reg_alpha=None, scale_pos_weight=None,
                              tree_method=None, validate_parameters=None,
                              verbosity=None),
    n_features_to_select=10)

In [498]:
names = x.columns
columns=[]
for i in range(len(fited.support_)):
    if fited.support_[i]:
        columns.append(names[i])

print("Columns with predictive power:", columns )
columns = columns + ['high', 'low', 'volume', 'open']
columns

Columns with predictive power: ['open', 'low', 'volume_adi', 'volume_obv', 'volume_cmf', 'volume_nvi', 'volatility_atr', 'volatility_kcl', 'volatility_ui', 'momentum_kama']


['open',
 'low',
 'volume_adi',
 'volume_obv',
 'volume_cmf',
 'volume_nvi',
 'volatility_atr',
 'volatility_kcl',
 'volatility_ui',
 'momentum_kama',
 'high',
 'low',
 'volume',
 'open']

In [499]:
x_test_cropped = x_test[columns]
x_train_cropped = x_train[columns]
x_train_cropped
x_test_cropped = x_test_cropped.loc[:,~x_test_cropped.columns.duplicated()]
x_train_cropped = x_train_cropped.loc[:,~x_train_cropped.columns.duplicated()]
#

In [500]:
for k,v in classifiers_boosted.items():
    print("Calculate: ", k)
    train_model(v,x_train_cropped,y_train)
    predictions_train[k] = v.predict(x_train_cropped)
    score_train[k] = accuracy_score(y_train.values, predictions_train[k])
    predictions[k] = v.predict(x_test_cropped)
    score[k] = accuracy_score(y_test.values, predictions[k])
    print('Score train: ',  score_train[k] )
    print('Score: ',  score[k] )
    # print_conf_matrix(test_y, predictions[k], k)

filename_to_export = f'C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\results\\train_test\\result_test_s_{symbol}_{WINDOW}_{ROWS_TO_PREDICT}_{datetime.now().strftime("%d_%m_%Y %H_%M_%S")}.csv'
filename_to_export_train= f'C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\results\\train_test\\result_train_s_{symbol}_{WINDOW}_{ROWS_TO_PREDICT}_{datetime.now().strftime("%d_%m_%Y %H_%M_%S")}.csv'

headers = ["Classifier type", "Accuracy"]
score_df = pd.DataFrame(score.items(), columns=headers)
print(tabulate(score_df, headers, tablefmt="psql"))
score_df.to_csv(filename_to_export,mode='a', index=False, header=False)
score_df_train = pd.DataFrame(score_train.items(), columns=headers)
print(tabulate(score_df_train, headers, tablefmt="psql"))
score_df_train.to_csv(filename_to_export_train,mode='a', index=False, header=False)

Calculate:  SXGBClassifier 1
[23:28:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Score train:  0.9762237762237762
Score:  0.369140625
Calculate:  SXGBClassifier 2
[23:28:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Score train:  0.9960584869675779
Score:  0.396484375
Calculate:  SXGBClassifier 3
[23:28:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'm

In [501]:
x_train_cropped.columns

Index(['open', 'low', 'volume_adi', 'volume_obv', 'volume_cmf', 'volume_nvi',
       'volatility_atr', 'volatility_kcl', 'volatility_ui', 'momentum_kama',
       'high', 'volume'],
      dtype='object')

In [502]:
#
# model = xgb.XGBClassifier(nthread =-1,max_depth=14,n_estimators=1000,
#                           eta =0.2)
# model.fit(x_train,y_train)
# predicted_train = model.predict(x_train)
# predicted_test = model.predict(x_test)
# print("------------")
# print(f'max_depth: {14}')
# score_train['XGBClassifier'] = accuracy_score(y_train.values, predicted_train)
# score['XGBClassifier'] = accuracy_score(y_test.values, predicted_test)
# print(score_train['XGBClassifier'])
# print(score['XGBClassifier'])
# print("------------")
#

In [503]:
# model = xgb.sklearn.XGBRFClassifier(n_jobs=-1,max_depth=12,n_estimators =100,eta=0.4)
# model.fit(x_train,y_train)
# predicted_train = model.predict(x_train)
# predicted_test = model.predict(x_test)
# print("------------")
# print(f'eta: ')
# score_train['XGBRFClassifier'] = accuracy_score(y_train.values, predicted_train)
# score['XGBRFClassifier'] = accuracy_score(y_test.values, predicted_test)
# print(accuracy_score(y_train.values, predicted_train))
# print(accuracy_score(y_test.values, predicted_test))
# print("------------")